# Track Array States During Evolution of Pattern 0

Runs a trace on the painting of the star pattern in the [psychedelia listing](https://github.com/mwenge/psychedelia-listing)

In [1]:
colors_base = {
"00":"c64_black",  
"01":"c64_white",  
"02":"c64_red",    
"03":"c64_cyan",   
"04":"c64_purple", 
"05":"c64_green",  
"06":"c64_blue",   
"07":"c64_yellow", 
"08":"c64_orange", 
"09":"c64_brown",  
"0a":"c64_ltred",  
"0b":"c64_gray1",  
"0c":"c64_lightgray",  
"0d":"c64_ltgreen",
"0e":"c64_ltblue", 
"0f":"c64_gray3",  
}
colors = [v for k,v in colors_base.items()]
    
c64_to_rgb = {
"c64_black": "#000000",     
"c64_white": "#ffffff",     
"c64_red": "#880000",       
"c64_cyan":  "#aaffee",     
"c64_purple": "#cc44cc",    
"c64_green": "#00cc55",     
#"c64_blue":  "#0000aa",     
"c64_blue":  "#0088ff",     
"c64_yellow":  "#eeee77",   
"c64_orange":  "#dd8855",   
"c64_brown": "#664400",     
"c64_ltred": "#ff7777",  
"c64_gray1":  "#333333",    
"c64_ltgreen": "#aaff66",
"c64_ltblue":  "#0088ff",
"c64_lightgrey": "#bbbbbb", 
"c64_lightgray": "#bbbbbb", 
"c64_gray3": "#bbbbbb",                             
}


Run the following command:
```sh
~/Dev/vice-emu-code/vice/src/x64sc -directory ~/Dev/vice-emu-code/vice/data/ -moncommands painttrace.txt bin/psychedelia.prg 
```
Where `painttrace.txt` is:

```
log on
logname "PsychedeliaPattern0StateTrace.txt"
tr exec 091b
command 1 "mem d800 dbff; mem 09cb 0a8a"
```

Read in the ram history.

In [191]:
log_file = "PsychedeliaPattern0StateTrace.txt"
input_file = open(log_file,'r')

flatten = lambda l: [e for sublist in l for e in sublist]

ram_history = []
array_history = []
lines = input_file.readlines()
c = 0
for i in range(5, len(lines), 79):
    raw_ram = [l[9:60].split() for l in lines[i:i+64]]
    raw_ram = [int(v,16) & 0x0f for l in raw_ram for v in l]
    color_ram = [raw_ram[v:v+40] for v in range(0, len(raw_ram), 40)][:-2]
    ram_history += [color_ram]
    
    x_ram = flatten([l[9:60].split() for l in lines[i+64:i+66]])
    y_ram = flatten([l[9:60].split() for l in lines[i+68:i+70]])
    c_ram = flatten([l[9:60].split() for l in lines[i+72:i+74]])
    array_history += [[x_ram, y_ram, c_ram]]
    c += 1
#print(ram_history[0])

In [192]:

patterns = []
patterns += [ram_history]


Methods for culling the pixel data to just the volume that's occupied.

In [193]:
def firstNonZero(l):
    for i,e in enumerate(l):
        if e > 0:
            return i
    return i

def getLeftMostPixelForPattern(pattern):
    return min([firstNonZero(r) for scr in pattern for r in scr])

def getRightMostPixelForPattern(pattern):
    return len(pattern[0][0]) - min([firstNonZero(list(reversed(r)))
                     for scr in pattern for r in scr])

def rotatePatternToCols(pattern):
    pattern_cols = []
    for scr in pattern:
        col_arr = [[0 for i in range(len(scr))] for j in range(len(scr[0]))]
        for i,r in enumerate(scr):
            for j,v in enumerate(r):
                col_arr[j][i] = v
        pattern_cols += [col_arr]
    return pattern_cols

def cullPattern(pattern):
    first_left = getLeftMostPixelForPattern(pattern)
    last_right = getRightMostPixelForPattern(pattern)
    width = last_right - first_left

    pattern_as_cols = rotatePatternToCols(pattern)
    first_top = getLeftMostPixelForPattern(pattern_as_cols)
    last_bottom = getRightMostPixelForPattern(pattern_as_cols)
    height = last_bottom - first_top
    
    culled = [[r[first_left:last_right] 
              for r in scr[first_top:last_bottom]]
              for scr in pattern]
    return culled

culled = cullPattern(patterns[0])
pattern = patterns[0]

In [201]:
culled[6]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 5, 7, 7, 7, 5, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Write the color ram as latex tables

In [182]:
#;                               0    6    2    4      5    3     7      1  
#presetColorValuesArray  .BYTE BLACK,BLUE,RED,PURPLE,GREEN,CYAN,YELLOW,WHITE

color_index_map = {
    0:0,6:1,2:2,4:3,5:4,3:5,7:6,1:7
}



Write a series of png files one for each stage of the evolution of the pattern

In [223]:
colors = {
0:"c64_black",  
1:"c64_white",  
2:"c64_red",    
3:"c64_cyan",   
4:"c64_purple", 
5:"c64_green",  
6:"c64_blue",   
7:"c64_yellow", 
}

from PIL import Image, ImageColor

bit_array = ([['0' for i in range(8)]] 
            + [['0']+['1' for i in range(6)]+['0'] for j in range(6)] 
            + [['0' for i in range(8)]])

def paintChar(origin, pixels, cv):
    c, r = origin
    x_o, y_o = c*8, r*8
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "0":
                continue
            color = c64_to_rgb[colors[cv]]
            pixels[x_o+x, y_o+y] = ImageColor.getrgb(color)


cull_pattern = cullPattern(pattern)
for j, scr in enumerate(cull_pattern):
    cols=len(scr[0])
    rows=len(scr)
    bits=8
    img = Image.new( 'RGB', (cols*bits,rows*bits), "black")
    pixels = img.load()
    for y, l in enumerate(scr):
        for x, v in enumerate(l):
            paintChar((x,y), pixels, v)
    img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
    img.save(f"../src/patterns/maps/pixel_pattern_{j}.png")



In [249]:
header = """
\\begin{figure}[H]                                                              
  {                                                                            
    \setlength{\\tabcolsep}{3.0pt}                                              
    \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule =                
    \\begin{adjustbox}{width=10cm,center}                                       
      \\footnotesize                                                            
      \\begin{tabular}{ll}"""

byte_cell = """        \makecell[l]{                                                          
        %BYTES%"""

pic_pattern = "\includegraphics[width=1.3cm]{src/patterns/maps/pixel_pattern_%NUM%.png}%" 

pic_cell = """        } & \makecell[l]{                                                      
            %PIC_PATTERN%
        } \\\\                                                                   
        \midrule"""

trailer = """
      \end{tabular}                                                            
    \end{adjustbox}                                                            
  }                                                                            
\end{figure}                                                                   
"""

pgroups = []
for i,(_,_,cs) in enumerate(array_history):
    ast = '\icode{currentColorIndexArray} Bytes 26-32:\\\\\n'
    ast += "        \icode{.BYTE  \$" + ',\$'.join([c.upper() for c in cs[25:]]) + "}\\\\"
    if len(pgroups) and pgroups[-1][0][0] == ast:
        pgroups[-1] += [(ast,i)]
    else:
        pgroups += [[(ast,i)]]
print(header)
for p in pgroups:
    byte_text = byte_cell.replace("%BYTES%", p[0][0])
    print(byte_text)
    pic_strings = "\n".join([pic_pattern.replace("%NUM%",str(n)) for _,n in p])
    pic_text = pic_cell.replace("%PIC_PATTERN%", pic_strings)
    print(pic_text)
print(trailer)


\begin{figure}[H]                                                              
  {                                                                            
    \setlength{\tabcolsep}{3.0pt}                                              
    \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule =                
    \begin{adjustbox}{width=10cm,center}                                       
      \footnotesize                                                            
      \begin{tabular}{ll}
        \makecell[l]{                                                          
        \icode{currentColorIndexArray} Bytes 26-32:\\
        \icode{.BYTE  \$FF,\$FF,\$06,\$FF,\$FF,\$FF,\$FF}\\
        } & \makecell[l]{                                                      
            \includegraphics[width=1.3cm]{src/patterns/maps/pixel_pattern_0.png}%
        } \\                                                                   
        \midrule
        \makecell[l]{                   

Create content for latex tables showing detail of each pattern with indices embedded.

In [225]:
colors = {
0:"c64_white",  
1:"c64_white",  
2:"c64_red",    
3:"c64_cyan",   
4:"c64_purple", 
5:"c64_green",  
6:"c64_blue",   
7:"c64_yellow", 
}

template = open('../src/patterns/maps/pattern0_ram_stage_template.tex','r').read()
for i, pt in enumerate(culled):
    chars = ""
    for l in pt:
        chars += ('\n' + ' & '.join([f"\\icode{{{str(color_index_map[c])}}}" for c in l]) + " \\\\")

    fills = ""
    for y,l in enumerate(reversed(pt)):
        for x,v in enumerate(l):
            if v == "01":
                continue
            fills += (f"\\fill[{colors[v]}] ({x},{y}) rectangle ++ (1,1);\n")
    output = template
    output = output.replace("%COLOR_MAP%", fills)
    output = output.replace("%RAM_MAP%", chars)
    output_file = open(f'../src/patterns/maps/pattern0_ram_stage{i}.tex', 'w')
    output_file.write(output)
    output_file.flush()
    _,_,cs = array_history[i]
    print(i, "\icode{.BYTE ... \$" + ',\$'.join([c.upper() for c in cs[25:]]) + "}\\\\")
    

0 \icode{.BYTE ... \$FF,\$FF,\$06,\$FF,\$FF,\$FF,\$FF}\\
1 \icode{.BYTE ... \$FF,\$FF,\$05,\$07,\$FF,\$FF,\$FF}\\
2 \icode{.BYTE ... \$FF,\$FF,\$05,\$07,\$FF,\$FF,\$FF}\\
3 \icode{.BYTE ... \$FF,\$FF,\$04,\$06,\$FF,\$FF,\$FF}\\
4 \icode{.BYTE ... \$FF,\$FF,\$04,\$06,\$07,\$FF,\$FF}\\
5 \icode{.BYTE ... \$FF,\$FF,\$04,\$06,\$07,\$FF,\$FF}\\
6 \icode{.BYTE ... \$FF,\$FF,\$03,\$06,\$07,\$FF,\$FF}\\
7 \icode{.BYTE ... \$FF,\$FF,\$03,\$05,\$06,\$FF,\$FF}\\
8 \icode{.BYTE ... \$FF,\$FF,\$03,\$05,\$06,\$FF,\$FF}\\
9 \icode{.BYTE ... \$FF,\$FF,\$03,\$05,\$06,\$FF,\$FF}\\
10 \icode{.BYTE ... \$FF,\$FF,\$03,\$05,\$06,\$FF,\$FF}\\
11 \icode{.BYTE ... \$FF,\$FF,\$02,\$05,\$06,\$FF,\$FF}\\
12 \icode{.BYTE ... \$FF,\$FF,\$02,\$05,\$06,\$FF,\$FF}\\
13 \icode{.BYTE ... \$FF,\$FF,\$02,\$04,\$06,\$07,\$FF}\\
14 \icode{.BYTE ... \$FF,\$FF,\$02,\$04,\$05,\$07,\$FF}\\
15 \icode{.BYTE ... \$FF,\$FF,\$02,\$04,\$05,\$07,\$FF}\\
16 \icode{.BYTE ... \$FF,\$FF,\$02,\$04,\$05,\$07,\$FF}\\
17 \icode{.BYTE ... \$FF

In [214]:
culled[7]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 5, 7, 1, 7, 5, 5, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]